In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import spacy
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import networkx as nx
from sklearn.neighbors import NearestNeighbors

In [2]:
dtype_dict = {3: 'str'}
df = pd.read_csv('dataset/amazon_data.csv', dtype=dtype_dict, low_memory=False)
df.head(3)

,Unnamed: 0.1,Unnamed: 0,title,original_price,sale_price,rating,review_count,main_category,sub_category_1,sub_category_2,...,is_available,predicted_trendiness,predicted_uniqueness,trendiness_score,uniqueness_score,ecommerce_text,product_embedding,Deliver to Singapore,Deliver to Malaysia,Delivery time
0,0,0,"Muslin Nursing Cover for Breastfeeding, Breath...",$12.99,$11.99,4.6,39,Baby Products,Baby,Privacy Nursing Covers,...,In Stock,0.346864,0.865101,0.159482,0.397760,passage: muslin nursing cover for breastfeedin...,"[0.020546678453683853, 0.04206167161464691, 0....",yes,no,10
1,1,1,Baby Sun Hat UPF 50+ Sun Protective Toddler Bu...,NaN,NaN,4.8,1739,Baby Products,"Clothing, Shoes & Jewelry",Baby Boys' Hats & Caps,...,Not Available,0.442018,0.947105,0.300501,0.719134,passage: baby sun hat upf sun protective todd...,"[0.02578025683760643, 0.045276544988155365, 0....",no,no,14
2,2,2,Dr. Brown's Milestones Wide-Neck Sippy Spout B...,NaN,$8.99,4.7,16812,Baby Products,Baby,Baby Bottles,...,In Stock,0.476247,0.886945,0.465965,0.483366,passage: dr browns milestones wideneck sippy s...,"[0.01611621305346489, 0.05096699669957161, 0.0...",yes,yes,6


In [3]:
import nltk
nltk.download('stopwords')

nlp = spacy.load("en_core_web_sm")
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
""""
def get_tfidf(product_details):
    clean_product = []
    product_name = list(product_details)
    for i in range(len(product_name)):
        words = ""

        doc = nlp(product_name[i].lower())
        for token in doc:
            token.lemma_ = re.sub(r'\W',' ',token.lemma_)
            token.lemma_ = token.lemma_.strip()
            if not token.lemma_.endswith("ml") and not token.lemma_.endswith("ms") and not token.lemma_.isdigit() and not token.lemma_ in stop_words:
                if len(token.lemma_) > 2 or token.lemma_ == 'uv': 
                    words += token.lemma_.lower() + " "
                    

        if len(words) > 0:
            clean_product.append(str(words.strip()))

    tfidf_vectorizer=TfidfVectorizer(use_idf=True) 
    tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(clean_product)
    first_vector_tfidfvectorizer=tfidf_vectorizer_vectors[0]

    df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=tfidf_vectorizer.get_feature_names(), 
                      columns=["tfidf"]) 
    df = df.sort_values(by=["tfidf"], ascending=False).reset_index()
    
    return df
    """
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

def get_tfidf(documents):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_vectorizer_vectors = tfidf_vectorizer.fit_transform(documents)
    first_vector_tfidfvectorizer = tfidf_vectorizer_vectors[0]
    
    df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), 
                      index=tfidf_vectorizer.get_feature_names_out(), 
                      columns=["tfidf"]) 
    df = df.sort_values(by=["tfidf"], ascending=False).reset_index()
    
    return df

In [20]:
G = nx.Graph()
# entries = os.listdir('dataset/updated/') ## Please Change Directory
main_category_list = []
sub_category_list = []
sub_category_list_2 = []
product_name_list = []

df['main_category'] = df['main_category'].str.lower()
df['sub_category_1'] = df['sub_category_1'].str.lower()
df['sub_category_2'] = df['sub_category_2'].str.lower()

main_category = df['main_category'].unique()
for _main_category in main_category:
    if type(_main_category) == str:
        print(_main_category)
        main_category_data = df[df['main_category'] == _main_category]
        G.add_node(_main_category)
        main_category_list.append(_main_category)
        
        for row in main_category_data['sub_category_1'].unique():
            if type(row) == str:
                sub_category_list.append(row)
                G.add_edge(row, _main_category, weight=1.0)
                
                for row2 in main_category_data.loc[main_category_data['sub_category_1'] == row]['sub_category_2'].unique():
                    if type(row2) == str:
                        G.add_edge(row2.strip(), row.strip(), weight=1.0)
                        sub_category_list_2.append(row2.strip())
                        
                        tfidf_result = get_tfidf(main_category_data.loc[main_category_data['sub_category_2'] == row2]['title'])
                        
                        index =  tfidf_result['index']
                        tfidf = tfidf_result['tfidf']
                        counter = 0
                        for _tfidf_result in tfidf_result['index']:
                            if float(tfidf[counter]) > 0.0:                    
                                if index[counter] not in main_category_list:
                                    product_name_list.append(index[counter].lower())
                                    G.add_edge(index[counter].lower(), row2.strip(), weight=tfidf[counter])
    
    
                            counter +=1

sports and outdoors
electronics
home and kitchen
computers
beauty and personal care
baby products
boy's fashion
girl's fashion
mens's fashion
main_category
toys and games
women's fashion


In [2]:
import pickle

#generate
#with open('network_theory.pickle','wb') as fe_data_file:
#     pickle.dump(G, fe_data_file)

#load
with open('dataset/network_theory.pickle', 'rb') as fe_data_file:
    G = pickle.load(fe_data_file)


In [3]:
import json
BETWEENNESS_CENTRALITY = nx.betweenness_centrality(G)
with open('dataset/betweenness_centrality.json', 'w') as json_file:
    json.dump(BETWEENNESS_CENTRALITY, json_file)

KeyboardInterrupt: 

In [4]:
with open('dataset/betweenness_centrality.json', 'r') as json_file:
    BETWEENNESS_CENTRALITY = json.load(json_file)

In [7]:
# BETWEENNESS_CENTRALITY = nx.betweenness_centrality(G)

#with open('betweeness_centrality.pickle','wb') as fe_data_file:
#     pickle.dump(G, fe_data_file)


# Load the betweenness centrality from the pickle file
with open('dataset/betweeness_centrality.pickle', 'rb') as fe_data_file:
    BETWEENNESS_CENTRALITY = pickle.load(fe_data_file)

In [83]:
def recommend_products(given_wishlist, G, BETWEENNESS_CENTRALITY, overall_data):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(given_wishlist.strip())
    sub_category_list_2 = list(overall_data['sub_category_2'].str.lower().unique())
    result_categories = set()  # Use a set to avoid duplicates

    for token in doc:
        if token.text in G.nodes():
            neighbors = [neighbor for neighbor in G.neighbors(token.text) if neighbor in sub_category_list_2]
            
            if neighbors:
                # Sort neighbors by betweenness centrality for lenient inclusion
                neighbors.sort(key=lambda x: -BETWEENNESS_CENTRALITY[x])
                
                # Select top N neighbors to include more products
                top_neighbors = neighbors[:10]  # Adjust the number of top neighbors as needed
                
                for neighbor in top_neighbors:
                    result_categories.add(neighbor)
    
    # Convert set to list to ensure more categories are included
    result_categories = list(result_categories)

    # If too few categories are found, expand to more neighbors
    if len(result_categories) < 50:  # Adjust this threshold as needed
        for token in doc:
            if token.text in G.nodes():
                additional_neighbors = [neighbor for neighbor in G.neighbors(token.text) if neighbor not in result_categories]
                
                for neighbor in additional_neighbors:
                    if neighbor in sub_category_list_2:
                        result_categories.add(neighbor)
                    if len(result_categories) >= 50:
                        break
            if len(result_categories) >= 50:
                break
    
    # Collect all products from the selected categories
    merge_products = [overall_data[overall_data['sub_category_2'] == result_category.title()] for result_category in result_categories]
    selected_category = pd.concat(merge_products).reset_index(drop=True)
    
    return selected_category

In [104]:
#Example usage
given_wishlist = 'gaming'
overall_data = pd.read_csv('dataset/product_sample.csv')  # Load your overall data
selected_category = recommend_products(given_wishlist, G, BETWEENNESS_CENTRALITY, overall_data)
selected_category

,Unnamed: 0.1,title,original_price,sale_price,rating,review_count,main_category,sub_category_1,sub_category_2,rankings,...,purchase_cnt_prev_month,store_name,is_available,predicted_trendiness,predicted_uniqueness,ecommerce_text,product_embedding,deliver_to_singapore,deliver_to_malaysia,delivery_time
0,2034,dell vostro 3910 full size tower business desk...,NaN,$499.00,4.5,27,Computers,Computers & Accessories,Tower Computers,7535,...,50,Dell Store,Only 6 left in stock - order soon.,0.306454,0.821096,passage: dell vostro full size tower business...,"[0.0077638751827180386, 0.044374946504831314, ...",no,yes,7
1,2051,"asus 2023 rog g22ch db776 gaming desktop pc, s...","$1,899.99","$1,749.99",4.5,14,Computers,Computers & Accessories,Tower Computers,3831,...,0,ASUS Store,Only 2 left in stock - order soon.,0.304812,0.821549,passage: asus rog gch db gaming desktop pc sm...,"[0.007109601981937885, 0.04614097252488136, 0....",no,yes,3
2,2069,ibuypower trace 7 mesh gaming pc computer desk...,NaN,$799.99,4.6,14,Computers,Computers & Accessories,Tower Computers,2786,...,100,iBUYPOWER Store,In Stock,0.349939,0.864313,passage: ibuypower trace mesh gaming pc compu...,"[0.010959627106785774, 0.043974295258522034, 0...",yes,yes,14
3,2093,ibuypower tracemesh gaming pc computer desktop...,NaN,$699.99,3.8,8,Computers,Computers & Accessories,Tower Computers,12002,...,0,iBUYPOWER Store,In Stock,0.057372,0.259535,passage: ibuypower tracemesh gaming pc compute...,"[0.01074367854744196, 0.04631184786558151, 0.0...",no,yes,10
4,2168,lenovo ideacentre gaming 5i - 2022 - gaming de...,$807.99,$649.99,4.3,49,Computers,Computers & Accessories,Tower Computers,3953,...,0,Lenovo Store,In Stock,0.220947,0.700440,passage: lenovo ideacentre gaming i gaming ...,"[0.01579936593770981, 0.04067809507250786, 0.0...",no,yes,9
5,2211,"stgaubron gaming desktop pc, intel xeon e5 3.6...",NaN,$429.99,3.9,42,Computers,Computers & Accessories,Tower Computers,15719,...,0,STGAubron Store,In Stock,0.086613,0.362984,passage: stgaubron gaming desktop pc intel xeo...,"[0.0011620003497228026, 0.045308664441108704, ...",no,yes,4
6,2322,"skytech o11 gaming pc, ryzen 7 7800x3d 4.2 ghz...",NaN,"$1,999.99",5.0,1,Computers,Computers & Accessories,Tower Computers,29355,...,0,Skytech Gaming Store,Only 7 left in stock (more on the way).,0.530197,1.000000,passage: skytech o gaming pc ryzen xd ghz gh...,"[0.017271896824240685, 0.043938394635915756, 0...",yes,yes,13
7,2473,"snowkids cat 8 ethernet cable 50 ft, flat high...",NaN,$19.99,4.7,3825,Computers,Cat 7 Ethernet Cables,Cat 7 Ethernet Cables,10,...,2000,Snowkids Store,In Stock,0.469158,0.887940,passage: snowkids cat ethernet cable ft flat...,"[0.016736328601837158, 0.0349062979221344, 0.0...",yes,no,3
8,2136,gorilla grip silky gel memory foam wrist rest ...,$19.99,$11.79,4.5,4137,Computers,Office Products,Wrist Rests,327,...,5000,Gorilla Grip Store,In Stock,0.485284,0.766958,passage: gorilla grip silky gel memory foam wr...,"[0.009778821840882301, 0.04310765117406845, 0....",yes,yes,12
9,2019,jabra evolve 65 se link380a ms stereo- bluetoo...,$131.24,$106.99,3.9,288,Computers,Computer Headsets,Computer Headsets,29,...,500,Jabra Store,Only 3 left in stock - order soon.,0.101261,0.355038,passage: jabra evolve se linka ms stereo blue...,"[0.018007103353738785, 0.05538419634103775, 0....",no,no,8


In [91]:
"""
def calculate_relevance(sample_wishlist, selected_category):
    vectorize = TfidfVectorizer(stop_words='english')
    tfidf_response = vectorize.fit_transform(selected_category['title'])
    dtm = pd.DataFrame(tfidf_response.todense(), columns=vectorize.get_feature_names())
    
    nn = NearestNeighbors(n_neighbors=selected_category.shape[0])
    nn.fit(dtm)
    
    new = vectorize.transform(sample_wishlist)
    knn_model_result = nn.kneighbors(new.todense())
    
    knn_result = pd.DataFrame(knn_model_result[0][0], columns=['Distance'])
    knn_result["title"] = selected_category['title'].iloc[knn_model_result[1][0]]
    
    merged_result = pd.merge(selected_category, knn_result, on='title', how='inner')
    merged_result = merged_result.drop_duplicates(subset='title', keep="first")
    
    return merged_result
"""
#Example usage function
def calculate_relevance(wishlist, category_data):
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_response = vectorizer.fit_transform(category_data['title'])
    dtm = pd.DataFrame(tfidf_response.todense(), columns=vectorizer.get_feature_names_out())
    
    nn = NearestNeighbors(n_neighbors=category_data.shape[0])
    nn.fit(dtm)
    
    wishlist_tfidf = vectorizer.transform(wishlist)
    distances, indices = nn.kneighbors(wishlist_tfidf)
    
    final_recommendations = category_data.iloc[indices[0]]
    
    return final_recommendations

In [93]:
#Example usage
sample_wishlist = ['glow']
final_recommendations = calculate_relevance(sample_wishlist, selected_category)
final_recommendations

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,Unnamed: 0.1,title,original_price,sale_price,rating,review_count,main_category,sub_category_1,sub_category_2,rankings,...,purchase_cnt_prev_month,store_name,is_available,predicted_trendiness,predicted_uniqueness,ecommerce_text,product_embedding,deliver_to_singapore,deliver_to_malaysia,delivery_time
4,5248,"Lulu Jr. Illustory Book Making Kit, Multicolor",$29.95,$29.95,4.7,3687,Toys and Games,Toys & Games,Kids' Drawing Kits,2727,...,1000,Lulu Jr.,In Stock,0.435746,0.895900,passage: lulu jr illustory book making kit mul...,"[0.03087785094976425, 0.026257438585162163, 0....",no,no,2
0,812,"Q-tips Cotton Swabs, 750 ct",$12.89,$5.37,4.8,11620,Beauty And Personal Care,Beauty & Personal Care,Cotton Swabs,597,...,10000,Q-tips Store,In Stock,0.824001,0.858842,passage: qtips cotton swabs ct q tips cotton ...,"[0.02027689293026924, 0.03857298195362091, 0.0...",yes,yes,10
1,856,"623181 Q-Tips Cotton Swab, Standard, White, 50...",$9.99,$7.39,4.8,45390,Beauty And Personal Care,Beauty & Personal Care,Cotton Swabs,1083,...,10000,Q-tips Store,In Stock,0.989852,0.817411,passage: qtips cotton swab standard white co...,"[0.016682753339409828, 0.03450196236371994, 0....",no,yes,10
2,5058,Skillmatics Art & Craft Activity - Poke-in Art...,$24.97,$19.97,4.8,39,Toys and Games,Toys & Games,Kids' Sewing Kits,878,...,1000,Skillmatics Store,In Stock,0.465562,0.941292,passage: skillmatics art craft activity poke...,"[0.005676414351910353, 0.0398731529712677, 0.0...",yes,no,7
6,5476,Decorate Your Own Water Bottle Kits for Girls ...,$21.99,$16.99,4.6,1459,Toys and Games,Toys & Games,Kids' Mosaic Kits,39,...,10000,EDsportshouse,In Stock,0.685991,0.777926,passage: decorate your own water bottle kits f...,"[0.02776952274143696, 0.040167391300201416, 0....",no,no,11
3,5159,"The Original Spirograph Deluxe Set, Arts and C...",$40.39,$23.95,4.7,10939,Toys and Games,Toys & Games,Kids' Drawing Kits,2843,...,1000,Kahootz Store,In Stock,0.467147,0.888791,passage: the original spirograph deluxe set ar...,"[0.02046377584338188, 0.03564508631825447, 0.0...",no,yes,3
5,5245,Dab-O-Ink 3oz Bingo Daubers - Mixed Colors - 12ct,$20.95,$17.79,4.7,7466,Toys and Games,Toys & Games,Bingo Daubers,2347,...,2000,Dab-O-Ink,In Stock,0.485031,0.884363,passage: daboink oz bingo daubers mixed color...,"[0.026414476335048676, 0.041911713778972626, 0...",no,yes,1
